In [ ]:
# Project info
PROJECT_ID = 'marvin-voice'
GIT_OWNER = 'patrikkj'
GIT_REPO = 'marvin-models'
DATASET = 'dataset' # either 'dataset' or 'dataset_full'

# Download code from Git
!git clone -q https://github.com/{GIT_OWNER}/{GIT_REPO}.git

# Download and unpack dataset from GCS bucket
!gsutil -q cp gs://{PROJECT_ID}/{DATASET}.zip /tmp
!unzip -q /tmp/{DATASET}.zip -d /content/{GIT_REPO}/data/{DATASET}

# Make scripts accessible within notebook
import sys
sys.path.append(f"/content/{GIT_REPO}")

In [5]:
# Project info
PROJECT_ID = 'marvin-voice'
DATASET = 'dataset' # either 'dataset' or 'dataset_full'

In [ ]:
# Download code from Git
GIT_OWNER = 'patrikkj'
GIT_REPO = 'marvin-models'
!git clone -q https://github.com/{GIT_OWNER}/{GIT_REPO}.git

In [6]:
# Download and unpack dataset from GCS bucket
PROJECT_ID = 'sheila-app'
DATASET = 'dataset' # either 'dataset' or 'dataset_full'
!gsutil cp gs://{PROJECT_ID}/{DATASET}.zip ./data
!unzip ./data/{DATASET}.zip -d ./data/{DATASET}

Copying gs://sheila-app/dataset.zip...
/ [0 files][    0.0 B/ 80.0 MiB]                                                
-
- [0 files][  3.1 MiB/ 80.0 MiB]                                                
\
\ [0 files][ 24.2 MiB/ 80.0 MiB]                                                
|
/
/ [0 files][ 44.9 MiB/ 80.0 MiB]                                                
-
\
\ [0 files][ 54.9 MiB/ 80.0 MiB]                                                
|
| [0 files][ 76.8 MiB/ 80.0 MiB]                                                
/
/ [1 files][ 80.0 MiB/ 80.0 MiB]                                                

Operation completed over 1 objects/80.0 MiB.                                     
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# Make scripts accessible within notebook
import sys
sys.path.append(f"/content/{GIT_REPO}")

In [7]:
!git -C marvin-models status


fatal: cannot change to 'marvin-models': No such file or directory


In [5]:
!gsutil

Usage: gsutil [-D] [-DD] [-h header]... [-m] [-o] [-q] [command [opts...] args...]
Available commands:
  acl              Get, set, or change bucket and/or object ACLs
  bucketpolicyonly Configure uniform bucket-level access
  cat              Concatenate object content to stdout
  compose          Concatenate a sequence of objects into a new composite object.
  config           Obtain credentials and create configuration file
  cors             Get or set a CORS JSON document for one or more buckets
  cp               Copy files and objects
  defacl           Get, set, or change default ACL on buckets
  defstorageclass  Get or set the default storage class on buckets
  du               Display object size usage
  hash             Calculate file hashes
  help             Get help about commands and topics
  hmac             CRUD operations on service account HMAC keys.
  iam              Get, set, or change bucket and/or object IAM permissions.
  kms              Configure Cloud KMS en

In [4]:
!dir

 Volume in drive J is Cloud
 Volume Serial Number is CA36-C4FE

 Directory of J:\Drive\Filer\Kode\git\marvin-models\notebooks

11.10.2020  15:37    <DIR>          .
11.10.2020  15:37    <DIR>          ..
11.10.2020  15:31    <DIR>          .ipynb_checkpoints
11.10.2020  15:36             3ÿ576 notebook.ipynb
10.10.2020  00:53                36 README.md
               2 File(s)          3ÿ612 bytes
               3 Dir(s)  15ÿ114ÿ461ÿ184 bytes free
